In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import os
import shutil
import glob
import numpy as np


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
pip install pandas==1.2.4

In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
!mkdir Dataset

I created two sub folders called 'cats' and 'dogs' in the dataset folder which i just created in the directory

In [ ]:
# I uploaded my cats and dogs images from my system into those files directly

In [ ]:
import tensorflow as tf


In [ ]:
tf.keras.applications.inception_v3.preprocess_input
from keras.applications.inception_v3 import preprocess_input

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import keras
from keras.layers import Dense, Flatten
from keras.models import Model 
from keras.preprocessing.image import ImageDataGenerator, load_img , img_to_array


In [ ]:
base_model=tf.keras.applications.inception_v3.InceptionV3(input_shape=(256,256,3),
    include_top=False
)

In [ ]:
for layer in base_model.layers:
  layer.trainable= False

In [ ]:
X= Flatten()(base_model.output)
X= Dense(units=2, activation= 'sigmoid')(X)

# Final model
model=Model(base_model.input , X)

# compile model
model.compile(optimizer= 'adam',loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

#summary
model.summary()

In [ ]:
## Use Data generator to carry out our image processing

In [ ]:
train_datagen=ImageDataGenerator(featurewise_center= True, rotation_range=0.4, 
                                 width_shift_range=0.3, 
                                 horizontal_flip=True, 
                                 preprocessing_function= preprocess_input ,
                                  zoom_range=0.4, 
                                 shear_range=0.4)

train_data=train_datagen.flow_from_directory(directory="/content/drive/MyDrive/Colab Notebooks/face_mask_images", 
                                             target_size=(256,256),
                                             batch_size=36)

In [ ]:
train_data.class_indices

In [ ]:
t_img , label= train_data.next()

In [ ]:
t_img.shape

In [ ]:
plotImages(t_img , label)

In [ ]:
#tRAIN THE MODEL

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc= ModelCheckpoint(filepath="./best_model.h5", 
                    monitor="accuracy", 
                    verbose=1, 
                    save_best_only=True)

es= EarlyStopping(monitor= "accuracy",
                  min_delta= 0.01,
                  patience=5,
                  verbose=1)

cb=[mc,es] 

In [ ]:
his =model.fit_generator(train_data, 
                         steps_per_epoch=10, 
                         epochs=30, 
                         callbacks=cb)

In [ ]:
from keras.models import load_model

model= load_model("/content/drive/MyDrive/Colab Notebooks/model/best_model.h5")

In [ ]:
h= his.history
h.keys()



In [ ]:
plt.plot(h['loss'])
plt.plot(h['accuracy'])

plt.title('LOSS vs ACC')
plt.show()

In [ ]:
h['loss']

In [ ]:
h['accuracy']

In [ ]:
#Validation

In [ ]:
path="/content/drive/MyDrive/Colab Notebooks/test_images/m00.jpg"
img= load_img(path, target_size=(256,256))

i= img_to_array(img)

i=preprocess_input(i)

input_arr=np.array([i])
input_arr.shape

pred=np.argmax(model.predict(input_arr))  

if pred == 0:
  print ("with_face_mask")  
else:
  print ("without_face_mask")

plt.imshow(input_arr[0])
plt.title("input image")
plt.show()  


In [ ]:
!pip install streamlit
#!pip install pyngrok

Grad-**CAM**

In [ ]:
%%writefile face_app.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image 
import requests
from skimage.transform import resize
from io import BytesIO
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.inception_v3 import preprocess_input



st.set_option('deprecation.showfileUploaderEncoding', False)
st.title('**Face Mask Recognition**')
st.text('***Upload your image here***')

@st.cache(allow_output_mutation=True)
def load_model():
  model= tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/model/best_model.h5')
  return model

model=load_model()

classes=['with_face_mask', 'without_face_mask']

uploaded_file=st.file_uploader("choose an image....", type=["jpg","png"])
if uploaded_file is not None:
  img= Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image') 

  if st.button('PREDICT'):
    classes=['with_face_mask', 'without_face_mask']
    img=img.resize((256,256))

    i= img_to_array(img)

    i=preprocess_input(i)

    input_arr=np.array([i])
    
    
    y_out=np.argmax(model.predict(input_arr))
    y_out=classes[y_out]
    
    st.write(f' I can confirm that this is ',y_out)


In [ ]:
#from pyngrok import ngrok

In [ ]:
#url=ngrok.connect(port='8501')
#url


In [ ]:
#!nohup streamlit run --server.port 80 face_app.py &>/dev/null

In [ ]:
!streamlit run face_app.py & npx localtunnel --port 8501